# Import libraries and setup¶

Oftentimes, the type of downstream analysis does not depend on the modality that was used to estimate cellular transitions - that’s why we modularized CellRank into kernels, which compute cell-cell transition matrices, and estimators, which analyze the transition matrices.

use CellRank kernels to compute a transition matrix of cellular dynamics.
use CellRank estimators to analyze the transition matrix, including the computation of fate probabilities, driver genes, and gene expression trends.
read and write CellRank kernels.

In [ ]:
# Import libraries we may need
import scanpy as sc
import numpy as np
import scipy as sp
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import scanpy.external as sce
import umap
reducer = umap.UMAP()
import scvelo as scv
import cellrank as cr
import anndata as ad
import igraph


In [ ]:
# Set up output figure settings
plt.rcParams['figure.figsize']=(128,128) #rescale figures, increase sizehere

# Set up scanpy settings
sc.settings.verbosity = 3
sc.set_figure_params(dpi=100, dpi_save=300) #Increase DPI for better resolution figures
#sc.logging.print_versions()

Global seed set to 0

In [ ]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)

Use these functions to either save or load the environmental variables, otherwise you will lose all the objects between opening/closing Jupyter sessions!

In [ ]:
#save the session
dill.dump_session('BM_Lonza_dataset_scVelo.db')

In [ ]:
#load the session
dill.load_session('DC_SCANPY.db')

# Read the data

In [ ]:
cd /Users/joaoluizsfilho/Library/CloudStorage/Dropbox/Work_Files/Matthias_Lab/Projects/Vivax_human_project/Results/Prospective_Study_1/BM_lonza/scRNAseq/Scanpy_analysis

In [ ]:
pwd

In [ ]:
adata = scv.read('./h5ad_files/Merged_dataset/BM_merged_query_scVelo_cellrank.h5ad', cache=True)

In [ ]:
adata.write('./h5ad_files/Merged_dataset/BM_merged_query_scVelo_cellrank.h5ad')

# 1- Working with kernels (if data already preprocessed)

1.1- Set up a kernel

To construct a transition matrix, CellRank offers a number of kernel classes in kernels. To demonstrate the concept, we’ll use the PseudotimeKernel which biases k-NN graph edges to point into the direction of increasing pseudotime, inspired by Palantir [Setty et al., 2019].

In [ ]:
adata

In [ ]:
from cellrank.kernels import PseudotimeKernel

pk = PseudotimeKernel(adata, time_key="palantir_pseudotime")

In [ ]:
# use this kernel to compute a cell-cell transition_matrix.
pk.compute_transition_matrix()

In [ ]:
pk

To get a first impression of the cellular dynamics in this dataset, we can simulate random walks on the Markov chain implied by the transition matrix, starting from hematopoietic stem cells (HSCs), and visualize these in the UMAP embedding.

Black and yellow dots indicate random walk start and terminal cells, respectively. Overall, this reflects the known differentiation hierachy in human hematopoiesis.

In [ ]:
pk.plot_random_walks(
    seed=0,
    n_sims=100,
    start_ixs={"SingleR.labels": "Hematopoietic stem cells_CD38- CD34+"},
    basis="X_proj.umap",
    legend_loc="right",
    dpi=150, save = ".pdf"
)

1.2- Kernel overview

There exist CellRank kernels for many different data modalities. Each modality and biological system comes with its own strenghts and limitations, so it’s important to choose the kernel carefully. We provide some guidance in the figure below. However, please check the kernel API for a complete and up-to-date list, as new kernels will come.

• Use the following kernels: VelocityKernel and PseudotimeKernel
    
Combining different kernels
Any two kernels can be globally combined via a weighted mean. 
To demonstrate this, let’s set up an additional cellrank.kernels.ConnectivityKernel, based on gene expression similarity.

In [ ]:
from cellrank.kernels import ConnectivityKernel

ck = ConnectivityKernel(adata).compute_transition_matrix()

In [ ]:
# Print the kernel to see some properties
ck

In [ ]:
# Combine with the PseudotimeKernel PseudotimeKernel from above.
# This works for any combination of kernels, and generalizes to more than two kernels. 
# Kernel combinations allow you to use distinct sources of information to describe cellular dynamics.
combined_kernel = 0.8 * pk + 0.2 * ck
combined_kernel

1.3- Writing and reading a kernel

Sometimes, we might want to write the kernel to disk, to either continue with the analysis later on, or to pass it to someone else. This can be easily done; simply write the kernel to adata, and write adata to file:

In [ ]:
pk.write_to_adata()
adata.write("BM_merged_query_scVelo_cellrank_pk.h5ad")

To continue with the analysis, we read the anndata.AnnData object from disk, and initialize a new kernel from the AnnData object.

In [ ]:
adata = sc.read("./h5ad_files/Merged_dataset/BM_merged_query_scVelo_cellrank_pk.h5ad")
pk = cr.kernels.PseudotimeKernel.from_adata(adata, key="T_fwd")
pk

In [ ]:
adata

# 2- Working with estimators

# 2.1- Set up an estimator

Estimators take a kernel object and offer methods to analyze it. The main objective is to decompose the state space into a set of macrostates that represent the slow-time scale dynamics of the process. A subset of these macrostates will be the initial or terminal states of the process, the remaining states will be intermediate transient states. CellRank currently offers two estimator classes in cellrank.estimators:

CFLARE: Clustering and Filtering Left And Right Eigenvectors. Heuristic method based on the spectrum of the transition matrix.
GPCCA: Generalized Perron Cluster Cluster Analysis: project the Markov chain onto a small set of macrostates using a Galerkin projection which maximizes the self-transition probability for the macrostates, [Reuter et al., 2019, Reuter et al., 2018].
We recommend the CFLARE estimator; let’s start by initializing it.

In [ ]:
from cellrank.estimators import GPCCA

g = GPCCA(pk)
print(g)

In [ ]:
g

# 2.2- Identify initial & terminal states

Fit the estimator to compute macrostates of cellular dynamics; these may include initial, terminal and intermediate states.

In [ ]:
g.fit(n_states=10, cluster_key="predicted.celltype.l2")

In [ ]:
g

In [ ]:
g.plot_macrostates(which="all", basis ="X_proj.umap", figsize = (8,8), save = 'macrostates.pdf')

For each macrostate, the algorithm computes an associated stability value. Let’s use the most stable macrostates as terminal states [Lange et al., 2022, Reuter et al., 2019, Reuter et al., 2018].

In [ ]:
g.predict_terminal_states(method="top_n", n_states=6)

In [ ]:
g.plot_macrostates(which="terminal", basis ="X_proj.umap", figsize = (8,8))

CellRank can also identify initial states - in this dataset, that does not make too much sense though, as the initial state was manually passed to Palantir to root the pseudotime computation.

# 2.3- Compute fate probabilities and driver genes

We can now compute how likely each cell is to reach each terminal state using compute_fate_probabilities().

In [ ]:
g.compute_fate_probabilities()

In [ ]:
g.plot_fate_probabilities(legend_loc="right", basis ="X_proj.umap", figsize = (8,8), save = 'predicted_terminal_states.png')

The plot above combines fate probabilities towards all terminal states, each cell is colored according to its most likely fate; color intensity reflects the degree of lineage priming. We could equally plot fate probabilities separately for each terminal state, or we can visualize them jointly in a circular projection [Lange et al., 2022, Velten et al., 2017].

Each dot represents a cell, colored by cluster labels. Cells are arranged inside the circle according to their fate probabilities; fate biased cells are placed next to their corresponding corner while naive cells are placed in the middle.

In [ ]:
cr.pl.circular_projection(adata, keys="predicted.celltype.l2", legend_loc="right", figsize = (32,8), save = 'circular_projection.pdf')

To infer putative driver genes for any of these trajectories, we correlate expression values with fate probabilities.

In [ ]:
mono_drivers = g.compute_lineage_drivers(lineages="Mono_1_1")
mono_drivers.head(10)

# 2.4- Visualize expression trends

Given fate probabilities and a pseudotime, we can plot trajectory-specific gene expression trends. Specifically, we fit Generalized Additive Models (GAMs), weighthing each cells contribution to each trajectory according to its vector of fate probabilities. We start by initializing a model.

The GAMR uses the R package mgcv for parameter fitting; this requires you to have rpy2 installed. To avoid this and remain entirely in python, you can use cellrank.models.GAM, which uses pyGAM for parameter fitting.

In [ ]:
model = cr.models.GAM(adata)

In [ ]:
adata.obs['SingleR.labels']

In [ ]:
model

We use MAGIC-imputed data for gene-trend visualization [van Dijk et al., 2018]. We don’t use imputed data for any other task.

In [ ]:
cr.pl.gene_trends(
    adata,
    model=model,
    data_key="MAGIC_imputed_data",
    genes=["GATA1", "TFRC", "IRF8", "MPO"],
    same_plot=True,
    ncols=1,
    time_key="palantir_pseudotime",
    hide_cells=True, legend_loc='best', figsize = (8,16), save = "gene_trends.pdf") #Valid options are: `'upper', 'top', 'center', 'lower', 'bottom'`

Above, we grouped expression trends by gene, and visualized several trajectories per panel. We can also do it the other way round - group expression trends by trajectory, visualize several genes per panel. While this is possible using the line plots from above (set transpose=True), we’ll demonstrate it using heatmaps.

In [ ]:
cr.pl.heatmap(
    adata,
    model=model,
    data_key="MAGIC_imputed_data",
    genes=["GATA1", "TFRC", "IRF8", "MPO"],
    lineages=['CD16 Mono', 'CD4 Naive', 'Late Eryth_2', 'Memory B_2', 'pDC', 'pre-mDC'],
    time_key="palantir_pseudotime",
    cbar=False,
    show_all_genes=True, figsize = (8, 4), save = "heatmap_gene_trends.pdf")